In [6]:
#import necessary dependency
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

# Defining a function that will detect face and predict if with mask or without mask
def facemask_detection_prediction(frame, MaskNetwork):
    # Resize the frame to have a maximum width of 400 pixels
    frame = imutils.resize(frame, width=400)

    # Preprocess the frame
    face = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face = cv2.resize(face, (224, 224))
    face = img_to_array(face)
    face = preprocess_input(face)
    face = np.expand_dims(face, axis=0)

    # Perform mask prediction on the preprocessed faces
    preds = MaskNetwork.predict(face)
    (mask, withoutMask) = preds[0]

    # Determine the class label and color
    label = "With Mask" if mask > withoutMask else "Without Mask"
    color = (0, 255, 0) if label == "With Mask" else (0, 0, 255)
    
    # Include the probability in the label
    label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

    return (label, color)

# Loading the mask detector model
MaskNetwork = load_model("mask_detector.model")

# Starting video stream
print("Starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

# Loop over the frames from the video stream
while True:
    # Getting the frame from the video stream
    frame = vs.read()

    # Detect faces in the frame and determine if they are wearing a face mask or not
    (label, color) = facemask_detection_prediction(frame, MaskNetwork)

    # Display the label and bounding box rectangle on the output frame
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    cv2.imshow("Frame", frame)
    
    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cv2.destroyAllWindows()
vs.stop()

Starting video stream...
1/1 [==============================] - 0s 171ms/step


1/1 [==============================] - 0s 150ms/step
